In [2]:
import numpy as np

print("this is a test")
print("updated from pycharm")

this is a test
updated from pycharm


In [ ]:
print("I am now adding something")
1+2

In [ ]:
df = np.zeros(3,3)
print(df)

In [ ]:
print("this is an update")

print("thiss")


In [ ]:
print("those changes ere made with Jupyter")
print("1")